In [78]:
# Libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from numpy import sqrt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, PowerTransformer, KBinsDiscretizer

In [79]:
# URLs for the datasets
url_red = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
url_white = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

# Loading the datasets
import pandas as pd

wine_quality_red = pd.read_csv(url_red, sep=';')
wine_quality_white = pd.read_csv(url_white, sep=';')

# Adding a 'type' column to distinguish between red and white wines
wine_quality_red['type'] = 'red'
wine_quality_white['type'] = 'white'

# Combine the datasets into one dataframe
data = pd.concat([wine_quality_red, wine_quality_white], ignore_index=True)

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [88]:

# EDA
#print(data.describe())
print(data.info())
#print(data.head(5))
#print(data.tail(5))
#len(data)

# Check for missing values
#missing_values = data.isnull().sum()
#print(missing_values)
#any_missing = data.isnull().any()
#print(any_missing)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  type                  6497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB
None


In [81]:
# Split dataset into training and & testing sets
X = data.drop('quality', axis=1)
y = data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)


In [82]:
"""
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.drop('quality')
categorical_features = ['type']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])
"""

"\n# feature engineering and creating pipeline\nnumeric_features = data.select_dtypes(include=['int64', 'float64']).columns.drop('quality')\ncategorical_features = ['type']\n\nnumeric_transformer = Pipeline(steps=[\n    ('imputer', SimpleImputer(strategy='median')),\n    ('scaler', StandardScaler())])\n\ncategorical_transformer = Pipeline(steps=[\n    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),\n    ('onehot', OneHotEncoder(handle_unknown='ignore'))])\n\npreprocessor = ColumnTransformer(\n    transformers=[\n        ('num', numeric_transformer, numeric_features),\n        ('cat', categorical_transformer, categorical_features)])\n\npipeline = Pipeline(steps=[('preprocessor', preprocessor),\n                           ('regressor', LinearRegression())])\n"

In [83]:
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.drop('quality')
categorical_features = ['type']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler()),
    ('power_transform', PowerTransformer(method='yeo-johnson'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

discretization_features = ['residual sugar', 'chlorides']
discretization_transformer = Pipeline(steps=[
    ('discretizer', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('discrete', discretization_transformer, discretization_features)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


In [84]:
# Fiting the pipeline and evaluating the models predictions and accuracy
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluation
rmse = sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R^2: {r2}')


RMSE: 0.7201627003619304
R^2: 0.2785949449846681


In [85]:
# Cross-validation to evaluate the pipeline's performance
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error')

# Mean of the cross-validation scores
cv_rmse = sqrt(-mean(cv_scores))

print(f'Cross-Validation RMSE: {cv_rmse}')


Cross-Validation RMSE: 0.7514394663845124
